In [2]:
import redis
import json
import time
from datetime import datetime, timedelta

redis_client = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

try:
    redis_client.ping()
    print("✅ Conexión a Redis establecida")
except Exception as e:
    print(f"❌ Error: {e}")


✅ Conexión a Redis establecida


In [3]:
def crear_notificacion(user_id, tipo, mensaje, ttl_segundos=3600):
    key = f"notifications:{user_id}"
    notificacion = {
        "id": f"notif_{int(time.time())}",
        "type": tipo,
        "message": mensaje,
        "timestamp": datetime.now().isoformat()
    }
    redis_client.lpush(key, json.dumps(notificacion))
    redis_client.expire(key, ttl_segundos)
    return notificacion

def obtener_notificaciones(user_id):
    key = f"notifications:{user_id}"
    notificaciones_json = redis_client.lrange(key, 0, -1)
    return [json.loads(n) for n in notificaciones_json]

def eliminar_notificaciones(user_id):
    key = f"notifications:{user_id}"
    return redis_client.delete(key)

def verificar_ttl(user_id):
    key = f"notifications:{user_id}"
    return redis_client.ttl(key)

print("✅ Funciones definidas")



✅ Funciones definidas


In [4]:
import time

print("=" * 60)
print("Velocidad de Redis")
print("=" * 60)

num_notificaciones = 10000
test_user = "test_user_performance"

start_time = time.time()

for i in range(num_notificaciones):
    crear_notificacion(
        test_user,
        "logro",
        f"Notificación de prueba #{i}",
        ttl_segundos=3600
    )

end_time = time.time()
tiempo_total = (end_time - start_time) * 1000
tiempo_por_operacion = tiempo_total / num_notificaciones

print(f"\n📊 Resultados:")
print(f"   • {num_notificaciones} notificaciones creadas")
print(f"   • Tiempo total: {tiempo_total:.2f} ms")
print(f"   • Tiempo por operación: {tiempo_por_operacion:.4f} ms")
print(f"   • Operaciones por segundo: {1000/tiempo_por_operacion:.0f} ops/seg")

eliminar_notificaciones(test_user)



Velocidad de Redis

📊 Resultados:
   • 10000 notificaciones creadas
   • Tiempo total: 7313.57 ms
   • Tiempo por operación: 0.7314 ms
   • Operaciones por segundo: 1367 ops/seg


1

In [5]:
print("=" * 60)
print("TTL Automático")
print("=" * 60)

ttl_user = "ttl_demo_user"

crear_notificacion(
    ttl_user,
    "recordatorio",
    "Esta notificación expirará en 5 segundos",
    ttl_segundos=5
)

print("\n1 Notificación creada con TTL de 5 segundos")
notificaciones = obtener_notificaciones(ttl_user)
print(f"   Notificaciones: {len(notificaciones)}")

ttl_restante = verificar_ttl(ttl_user)
print(f"   TTL restante: {ttl_restante} segundos")

print("\n Esperando 6 segundos...")
time.sleep(6)

print("\n2 Después de 6 segundos:")
notificaciones = obtener_notificaciones(ttl_user)
print(f"   Notificaciones: {len(notificaciones)}")

ttl_restante = verificar_ttl(ttl_user)
if ttl_restante == -2:
    print("   ✅ Redis eliminó automáticamente la notificación (clave no existe)")

TTL Automático

1 Notificación creada con TTL de 5 segundos
   Notificaciones: 1
   TTL restante: 5 segundos

 Esperando 6 segundos...

2 Después de 6 segundos:
   Notificaciones: 0
   ✅ Redis eliminó automáticamente la notificación (clave no existe)


In [6]:
user_id = "duolingo_user_123"

crear_notificacion(user_id, "logro", "¡Felicidades! Has completado la Unidad 5 🎉", ttl_segundos=3600)
crear_notificacion(user_id, "recordatorio", "🔥 Llevas 7 días seguidos aprendiendo", ttl_segundos=3600)

print("=" * 60)
print("📱 FLUJO: Usuario abre la aplicación Duolingo")
print("=" * 60)

print("\n1- Dispositivo solicita notificaciones pendientes...")
notificaciones = obtener_notificaciones(user_id)
print(f"{len(notificaciones)} notificaciones encontradas")

print("\n2- Mostrando notificaciones al usuario:")
for notif in notificaciones:
    print(f"{notif['message']}")

print("\n3- Notificaciones entregadas exitosamente")
print("  Eliminando notificaciones de Redis...")
eliminadas = eliminar_notificaciones(user_id)

if eliminadas:
    print(" Notificaciones han sido eliminadas")

notificaciones_restantes = obtener_notificaciones(user_id)
print(f"\n4- Verificación: {len(notificaciones_restantes)} notificaciones restantes")



📱 FLUJO: Usuario abre la aplicación Duolingo

1- Dispositivo solicita notificaciones pendientes...
2 notificaciones encontradas

2- Mostrando notificaciones al usuario:
🔥 Llevas 7 días seguidos aprendiendo
¡Felicidades! Has completado la Unidad 5 🎉

3- Notificaciones entregadas exitosamente
  Eliminando notificaciones de Redis...
 Notificaciones han sido eliminadas

4- Verificación: 0 notificaciones restantes
